## Homework

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-08 15:58:51--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 108.177.14.194, 2a00:1450:4010:c0f::c2
Connecting to docs.google.com (docs.google.com)|108.177.14.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/v4c3mkouamn14audi6fca7o2l73i9d2o/1652014725000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-08 15:58:52--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/v4c3mkouamn14audi6fca7o2l73i9d2o/1652014725000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 173.194.222.132, 2a00:1450:4010:c0b::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleuserconten

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

In [43]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])

In [5]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 292 entries, id to price
dtypes: datetime64[ns](1), float64(119), int64(157), object(15)
memory usage: 44.6+ MB
None


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [9]:
print(df.columns)

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
       'cafe_count_5000_price_high', 'big_church_count_5000',
       'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
       'sport_count_5000', 'market_count_5000', 'price'],
      dtype='object', length=292)


Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

Выкинем пока колонки id и timestamp

In [44]:
df_dropped = df.drop(columns=["id", "timestamp"])

In [46]:
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology']
num_columns = list(set(df_dropped.columns).difference(set(cat_columns)))

Заполним пропуски в чиcловых признаках средним, а в категориальных - самым частым.

In [48]:
df_filled = pd.DataFrame(df_dropped, copy=True)
df_filled[num_columns] = SimpleImputer(strategy="mean").fit_transform(df_dropped[num_columns])
df_filled[cat_columns] = SimpleImputer(strategy="most_frequent").fit_transform(df_dropped[cat_columns])

In [49]:
df_filled

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.00000,1.000000,1.0,OwnerOccupier,...,36.0,7.0,2.0,15.0,33.0,1.0,12.0,75.0,10.0,15318960.0
1,64.0,64.000000,16.0,12.603928,1.0,3687.417575,1.91071,6.454917,2.0,OwnerOccupier,...,2.0,2.0,0.0,0.0,13.0,1.0,0.0,6.0,1.0,6080000.0
2,83.0,44.000000,9.0,17.000000,1.0,1985.000000,3.00000,10.000000,3.0,Investment,...,13.0,6.0,1.0,8.0,18.0,0.0,1.0,52.0,0.0,17000000.0
3,71.0,49.000000,2.0,12.603928,1.0,3687.417575,1.91071,6.454917,2.0,Investment,...,0.0,0.0,0.0,1.0,3.0,0.0,2.0,8.0,2.0,990000.0
4,60.0,42.000000,9.0,9.000000,1.0,1970.000000,3.00000,6.000000,2.0,Investment,...,3.0,1.0,0.0,5.0,8.0,0.0,1.0,34.0,5.0,7900000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45.0,27.000000,2.0,9.000000,1.0,1978.000000,2.00000,5.000000,2.0,Investment,...,9.0,0.0,0.0,6.0,29.0,0.0,1.0,46.0,10.0,7200000.0
19996,38.0,34.032333,4.0,17.000000,1.0,3687.417575,2.00000,1.000000,1.0,OwnerOccupier,...,2.0,2.0,0.0,1.0,12.0,1.0,0.0,8.0,0.0,3971520.0
19997,35.0,14.000000,9.0,22.000000,1.0,2001.000000,1.00000,10.000000,2.0,Investment,...,13.0,3.0,0.0,6.0,27.0,1.0,10.0,57.0,13.0,7200000.0
19998,51.0,30.000000,8.0,17.000000,1.0,2011.000000,2.00000,9.000000,2.0,Investment,...,1.0,0.0,0.0,0.0,7.0,0.0,0.0,3.0,0.0,3300000.0


Для категориальных признаков применим one-hot кодирование

In [50]:
df_enc = pd.get_dummies(df_filled)

In [53]:
df_enc

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ID_bus_terminal_10,ID_bus_terminal_11,ID_bus_terminal_12,ID_bus_terminal_13,ID_bus_terminal_14,ecology_excellent,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory
0,1.0,1.000000,1.0,1.000000,1.000000,1.00000,1.000000,5.293465e+06,77878.0,0.023464,...,0,0,0,0,0,0,0,0,1,0
1,64.0,64.000000,16.0,12.603928,3687.417575,1.91071,6.454917,6.677245e+07,9553.0,0.336177,...,0,0,0,0,0,0,0,1,0,0
2,83.0,44.000000,9.0,17.000000,1985.000000,3.00000,10.000000,1.216448e+07,78507.0,0.297166,...,0,0,0,0,0,0,1,0,0,0
3,71.0,49.000000,2.0,12.603928,3687.417575,1.91071,6.454917,4.708040e+06,38075.0,0.271702,...,0,0,0,0,0,0,0,1,0,0
4,60.0,42.000000,9.0,9.000000,1970.000000,3.00000,6.000000,1.428699e+07,157010.0,0.389354,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45.0,27.000000,2.0,9.000000,1978.000000,2.00000,5.000000,2.481385e+07,174831.0,0.683844,...,0,0,0,0,0,0,1,0,0,0
19996,38.0,34.032333,4.0,17.000000,3687.417575,2.00000,1.000000,6.677245e+07,9553.0,0.336177,...,0,0,0,0,0,0,0,1,0,0
19997,35.0,14.000000,9.0,22.000000,2001.000000,1.00000,10.000000,1.004686e+07,76156.0,0.071056,...,0,0,0,0,0,1,0,0,0,0
19998,51.0,30.000000,8.0,17.000000,2011.000000,2.00000,9.000000,4.036700e+07,21819.0,0.492840,...,0,0,0,0,0,0,0,1,0,0


In [52]:
X = df_enc.drop(columns=["price"])
y = df_enc["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [25]:
from sklearn.tree import DecisionTreeRegressor

In [54]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)

In [56]:
pred

array([11684305.,  5813760., 18000000., ...,  2357000.,  5550000.,
        6800000.])

Проверьте качество на отложенной выборке.

In [61]:
print(mean_squared_error(pred, y_test)**0.5)

3833207.632678462


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [63]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_enc["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_enc["week_year_cnt"] = week_year.map(week_year_cnt_map)

/tmp/ipykernel_139193/3601704785.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [70]:
df_enc["month"] = df.timestamp.dt.month
df_enc["week_day"] = df.timestamp.dt.weekday
df_enc["floor_to_max"] = df_enc["floor"] / df_enc["max_floor"]
df_enc["kitchen_to_full"] = df_enc["kitch_sq"] / df_enc["full_sq"]
df_enc["mean_room"] = df_enc["full_sq"] / df_enc["num_room"]
df_enc["life_to_full"] = df_enc["life_sq"] / df_enc["full_sq"]

Помимо оюязательных были также добавлены: средняяя площадь комнаты и доля жилой площади от общей.

In [71]:
df_enc

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,area_m,raion_popul,green_zone_part,...,ecology_poor,ecology_satisfactory,month_year_cnt,week_year_cnt,month,week_day,floor_to_max,kitchen_to_full,mean_room,life_to_full
0,1.0,1.000000,1.0,1.000000,1.000000,1.00000,1.000000,5.293465e+06,77878.0,0.023464,...,1,0,1122,112,12,4,1.000000,1.000000,1.000000,1.000000
1,64.0,64.000000,16.0,12.603928,3687.417575,1.91071,6.454917,6.677245e+07,9553.0,0.336177,...,0,0,327,68,10,3,1.269446,0.100858,33.495396,1.000000
2,83.0,44.000000,9.0,17.000000,1985.000000,3.00000,10.000000,1.216448e+07,78507.0,0.297166,...,0,0,752,192,2,2,0.529412,0.120482,27.666667,0.530120
3,71.0,49.000000,2.0,12.603928,3687.417575,1.91071,6.454917,4.708040e+06,38075.0,0.271702,...,0,0,262,69,7,3,0.158681,0.090914,37.158955,0.690141
4,60.0,42.000000,9.0,9.000000,1970.000000,3.00000,6.000000,1.428699e+07,157010.0,0.389354,...,0,0,711,214,10,2,1.000000,0.100000,20.000000,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45.0,27.000000,2.0,9.000000,1978.000000,2.00000,5.000000,2.481385e+07,174831.0,0.683844,...,0,0,734,181,9,0,0.222222,0.111111,22.500000,0.600000
19996,38.0,34.032333,4.0,17.000000,3687.417575,2.00000,1.000000,6.677245e+07,9553.0,0.336177,...,0,0,579,153,12,4,0.235294,0.026316,19.000000,0.895588
19997,35.0,14.000000,9.0,22.000000,2001.000000,1.00000,10.000000,1.004686e+07,76156.0,0.071056,...,0,0,861,277,6,3,0.409091,0.285714,35.000000,0.400000
19998,51.0,30.000000,8.0,17.000000,2011.000000,2.00000,9.000000,4.036700e+07,21819.0,0.492840,...,0,0,573,164,1,1,0.470588,0.176471,25.500000,0.588235


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [76]:
df_enc.isnull().any(axis=1).mean()

0.00015

In [88]:
np.array(df_enc == np.inf).any(axis=1).shape

(19997,)

In [78]:
df_enc = df_enc.dropna(axis=0)

In [91]:
m = df_enc.loc[df_enc["floor_to_max"] != np.inf, "floor_to_max"].max()
df_enc["floor_to_max"].replace(np.inf, m, inplace=True)
m = df_enc.loc[df_enc["kitchen_to_full"] != np.inf, "kitchen_to_full"].max()
df_enc["kitchen_to_full"].replace(np.inf, m, inplace=True)
m = df_enc.loc[df_enc["mean_room"] != np.inf, "mean_room"].max()
df_enc["mean_room"].replace(np.inf, m, inplace=True)
m = df_enc.loc[df_enc["life_to_full"] != np.inf, "life_to_full"].max()
df_enc["life_to_full"].replace(np.inf, m, inplace=True)

In [92]:
X = df_enc.drop(columns=["price"])
y = df_enc["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

### Дерево решений

In [93]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
print(mean_squared_error(pred, y_test)**0.5)

3630828.0567652574


Как видно, путем feature engineering удалось немного повысить качество предсказаний

In [100]:
tree = DecisionTreeRegressor()
parameters = {"max_depth":[2, 10, 50, 100, 200], "min_samples_leaf":[1, 5, 10, 100, 1000]}
gs_tree = GridSearchCV(tree,
                      param_grid=parameters,
                      scoring="neg_mean_squared_error",
                      cv=5)
gs_tree.fit(X_train, y_train)
gs_tree.best_params_

{'max_depth': 10, 'min_samples_leaf': 100}

In [104]:
tree = DecisionTreeRegressor(max_depth=10, min_samples_leaf=100)
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
print(mean_squared_error(pred, y_test)**0.5)

3039220.0369040035


За счет подбора параметров удалось значительно улучшить какчество

### Случайный лес

In [109]:
import multiprocessing
from sklearn.ensemble import RandomForestRegressor
workers = multiprocessing.cpu_count() - 1

In [118]:
forest = RandomForestRegressor(n_jobs=workers)
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_test)
print(mean_squared_error(forest_pred, y_test)**0.5)

2704279.3837902145


Как видно лес с параметрами по-умолчанию справляется сильно лучше одного дерева, попробуем перебрать некоторые параметры

In [116]:
forest = RandomForestRegressor(min_samples_leaf=100, random_state=42, n_jobs=workers)
parameters = {"n_estimators": [100, 200], "max_depth": [5, 10, 20], "max_features": [0.2, 0.333, "sqrt", "auto"], "max_samples": [0.1, 0.2, 0.5, None]}
gs_forest = GridSearchCV(forest,
                      param_grid=parameters,
                      scoring="neg_mean_squared_error",
                      cv=5)
gs_forest.fit(X_train, y_train)
gs_forest.best_params_

{'max_depth': 20,
 'max_features': 'auto',
 'max_samples': None,
 'n_estimators': 200}

In [117]:
forest = RandomForestRegressor(max_depth=20, max_features="auto", max_samples=None, n_estimators=200, min_samples_leaf=100, n_jobs=workers)
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_test)
print(mean_squared_error(forest_pred, y_test)**0.5)

2978504.8092333404


Как видно, среди параметров не оказалось, лучше тех, что по-умолчанию. Требуется более скрупулезный их перебор. В связи с довольно длительным обучением (и надвигающейся сессией) перебор был остановлен на этом.

### Бустинг

In [119]:
boost = CatBoostRegressor(loss_function="RMSE")
boost.fit(X_train, y_train)
boost_pred = boost.predict(X_test)
print(mean_squared_error(boost_pred, y_test)**0.5)

Learning rate set to 0.063448
0:	learn: 4464813.6744370	total: 100ms	remaining: 1m 40s
1:	learn: 4335308.7743433	total: 138ms	remaining: 1m 9s
2:	learn: 4215156.1026448	total: 159ms	remaining: 52.8s
3:	learn: 4096135.3991552	total: 175ms	remaining: 43.5s
4:	learn: 3982422.8985248	total: 188ms	remaining: 37.5s
5:	learn: 3877055.1840489	total: 204ms	remaining: 33.9s
6:	learn: 3786225.2167692	total: 220ms	remaining: 31.2s
7:	learn: 3700179.5261346	total: 235ms	remaining: 29.2s
8:	learn: 3621680.4191614	total: 248ms	remaining: 27.3s
9:	learn: 3548224.1121204	total: 260ms	remaining: 25.8s
10:	learn: 3480884.4152648	total: 273ms	remaining: 24.6s
11:	learn: 3417978.8744958	total: 286ms	remaining: 23.5s
12:	learn: 3359804.6055268	total: 299ms	remaining: 22.7s
13:	learn: 3306671.4970319	total: 312ms	remaining: 21.9s
14:	learn: 3257222.6091226	total: 325ms	remaining: 21.3s
15:	learn: 3213598.8505257	total: 337ms	remaining: 20.7s
16:	learn: 3171605.6789818	total: 351ms	remaining: 20.3s
17:	learn:

144:	learn: 2277898.2363531	total: 2.09s	remaining: 12.3s
145:	learn: 2277138.3751421	total: 2.1s	remaining: 12.3s
146:	learn: 2274360.5019269	total: 2.12s	remaining: 12.3s
147:	learn: 2273051.1630163	total: 2.13s	remaining: 12.3s
148:	learn: 2272239.9136236	total: 2.15s	remaining: 12.3s
149:	learn: 2271015.8679702	total: 2.16s	remaining: 12.2s
150:	learn: 2270248.8219173	total: 2.17s	remaining: 12.2s
151:	learn: 2268958.6412167	total: 2.18s	remaining: 12.2s
152:	learn: 2266558.6687247	total: 2.19s	remaining: 12.1s
153:	learn: 2265396.6243408	total: 2.21s	remaining: 12.1s
154:	learn: 2262131.2926231	total: 2.22s	remaining: 12.1s
155:	learn: 2259001.4500755	total: 2.23s	remaining: 12.1s
156:	learn: 2257149.2194877	total: 2.25s	remaining: 12.1s
157:	learn: 2256022.0456873	total: 2.26s	remaining: 12s
158:	learn: 2254400.3368010	total: 2.27s	remaining: 12s
159:	learn: 2251192.6018907	total: 2.28s	remaining: 12s
160:	learn: 2250725.0079410	total: 2.3s	remaining: 12s
161:	learn: 2249213.4451

289:	learn: 2063150.0865363	total: 4.16s	remaining: 10.2s
290:	learn: 2062702.2470974	total: 4.18s	remaining: 10.2s
291:	learn: 2061054.2175386	total: 4.19s	remaining: 10.2s
292:	learn: 2060149.2820246	total: 4.2s	remaining: 10.1s
293:	learn: 2058530.2268855	total: 4.22s	remaining: 10.1s
294:	learn: 2058410.6491687	total: 4.23s	remaining: 10.1s
295:	learn: 2057073.8578265	total: 4.24s	remaining: 10.1s
296:	learn: 2055766.7445431	total: 4.26s	remaining: 10.1s
297:	learn: 2054172.3800880	total: 4.27s	remaining: 10.1s
298:	learn: 2051840.4601716	total: 4.28s	remaining: 10s
299:	learn: 2050240.4211042	total: 4.31s	remaining: 10.1s
300:	learn: 2050113.4228706	total: 4.32s	remaining: 10s
301:	learn: 2049661.9210540	total: 4.34s	remaining: 10s
302:	learn: 2048374.5339620	total: 4.37s	remaining: 10s
303:	learn: 2048023.0619248	total: 4.39s	remaining: 10s
304:	learn: 2047113.9106561	total: 4.41s	remaining: 10s
305:	learn: 2045935.4980580	total: 4.42s	remaining: 10s
306:	learn: 2045826.9079742	t

440:	learn: 1924146.6804112	total: 6.4s	remaining: 8.11s
441:	learn: 1922931.3592354	total: 6.42s	remaining: 8.1s
442:	learn: 1922164.6328719	total: 6.43s	remaining: 8.09s
443:	learn: 1921008.8313784	total: 6.45s	remaining: 8.07s
444:	learn: 1920776.1911787	total: 6.46s	remaining: 8.05s
445:	learn: 1919986.2880805	total: 6.47s	remaining: 8.04s
446:	learn: 1919163.1000378	total: 6.48s	remaining: 8.02s
447:	learn: 1917856.5179211	total: 6.5s	remaining: 8.01s
448:	learn: 1917096.7659595	total: 6.51s	remaining: 7.99s
449:	learn: 1916142.1526769	total: 6.52s	remaining: 7.97s
450:	learn: 1915766.5022946	total: 6.54s	remaining: 7.96s
451:	learn: 1915699.3428991	total: 6.55s	remaining: 7.94s
452:	learn: 1914661.7212625	total: 6.56s	remaining: 7.92s
453:	learn: 1913942.6101020	total: 6.58s	remaining: 7.91s
454:	learn: 1913102.3770404	total: 6.59s	remaining: 7.89s
455:	learn: 1912918.1443216	total: 6.6s	remaining: 7.87s
456:	learn: 1911731.9497919	total: 6.62s	remaining: 7.86s
457:	learn: 191116

582:	learn: 1823213.8706348	total: 8.29s	remaining: 5.93s
583:	learn: 1822421.0026605	total: 8.31s	remaining: 5.92s
584:	learn: 1821940.4716019	total: 8.32s	remaining: 5.9s
585:	learn: 1821538.2060113	total: 8.33s	remaining: 5.89s
586:	learn: 1821046.2534045	total: 8.35s	remaining: 5.87s
587:	learn: 1820121.4124359	total: 8.36s	remaining: 5.86s
588:	learn: 1819376.0710659	total: 8.37s	remaining: 5.84s
589:	learn: 1818801.6131972	total: 8.39s	remaining: 5.83s
590:	learn: 1818135.0720002	total: 8.4s	remaining: 5.81s
591:	learn: 1816849.2698422	total: 8.41s	remaining: 5.8s
592:	learn: 1816586.1765183	total: 8.43s	remaining: 5.78s
593:	learn: 1816126.2720801	total: 8.44s	remaining: 5.77s
594:	learn: 1814984.5989709	total: 8.45s	remaining: 5.75s
595:	learn: 1814501.4473620	total: 8.46s	remaining: 5.74s
596:	learn: 1814380.2013777	total: 8.48s	remaining: 5.72s
597:	learn: 1814236.9609079	total: 8.49s	remaining: 5.71s
598:	learn: 1813541.2229034	total: 8.51s	remaining: 5.69s
599:	learn: 18129

724:	learn: 1728302.9540424	total: 10.4s	remaining: 3.95s
725:	learn: 1727117.4154471	total: 10.4s	remaining: 3.94s
726:	learn: 1726137.1520938	total: 10.5s	remaining: 3.92s
727:	learn: 1725579.1640145	total: 10.5s	remaining: 3.91s
728:	learn: 1725268.1251893	total: 10.5s	remaining: 3.9s
729:	learn: 1724993.4510235	total: 10.5s	remaining: 3.89s
730:	learn: 1724176.9197373	total: 10.5s	remaining: 3.88s
731:	learn: 1723615.9627134	total: 10.6s	remaining: 3.86s
732:	learn: 1722862.5336697	total: 10.6s	remaining: 3.85s
733:	learn: 1722389.8885309	total: 10.6s	remaining: 3.84s
734:	learn: 1721866.9874191	total: 10.6s	remaining: 3.83s
735:	learn: 1721190.8716892	total: 10.6s	remaining: 3.81s
736:	learn: 1720561.4722677	total: 10.6s	remaining: 3.8s
737:	learn: 1720009.7190761	total: 10.7s	remaining: 3.78s
738:	learn: 1719716.1637148	total: 10.7s	remaining: 3.77s
739:	learn: 1718969.8481639	total: 10.7s	remaining: 3.75s
740:	learn: 1718343.1044771	total: 10.7s	remaining: 3.74s
741:	learn: 1717

875:	learn: 1646387.9095986	total: 12.7s	remaining: 1.8s
876:	learn: 1645736.1943951	total: 12.7s	remaining: 1.79s
877:	learn: 1645486.7743354	total: 12.8s	remaining: 1.77s
878:	learn: 1644819.8352614	total: 12.8s	remaining: 1.76s
879:	learn: 1644527.7871616	total: 12.8s	remaining: 1.74s
880:	learn: 1643904.9038869	total: 12.8s	remaining: 1.73s
881:	learn: 1643355.3138603	total: 12.8s	remaining: 1.72s
882:	learn: 1642788.2565005	total: 12.8s	remaining: 1.7s
883:	learn: 1642207.1031234	total: 12.9s	remaining: 1.69s
884:	learn: 1641493.4804219	total: 12.9s	remaining: 1.67s
885:	learn: 1641107.5756304	total: 12.9s	remaining: 1.66s
886:	learn: 1640587.4591743	total: 12.9s	remaining: 1.64s
887:	learn: 1639901.9976029	total: 12.9s	remaining: 1.63s
888:	learn: 1639463.2529508	total: 12.9s	remaining: 1.61s
889:	learn: 1639138.7772356	total: 12.9s	remaining: 1.6s
890:	learn: 1638684.9672198	total: 13s	remaining: 1.58s
891:	learn: 1637995.5421709	total: 13s	remaining: 1.57s
892:	learn: 1637912.6

Как видно catboost с параметрвами по-умолчанию справляется лучше предыдущих моделей.

In [121]:
boost = CatBoostRegressor(loss_function="RMSE", random_state=42)
parameters = {"iterations": [10, 20, 100, 200], "learning_rate": [0.02, 0.05, 0.1], "depth": [3, 5, 7]}
result = boost.grid_search(parameters, X=X_train, y=y_train, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 8381361.9927859	test: 8169021.1839713	best: 8169021.1839713 (0)	total: 6.55ms	remaining: 59ms
1:	learn: 8248366.0438962	test: 8034644.4275664	best: 8034644.4275664 (1)	total: 12.9ms	remaining: 51.7ms
2:	learn: 8119232.3167760	test: 7905922.0816401	best: 7905922.0816401 (2)	total: 19.3ms	remaining: 45ms
3:	learn: 7993289.4507316	test: 7780828.6612690	best: 7780828.6612690 (3)	total: 30.3ms	remaining: 45.4ms
4:	learn: 7873026.6075462	test: 7664095.3963142	best: 7664095.3963142 (4)	total: 36.1ms	remaining: 36.1ms
5:	learn: 7752355.4526254	test: 7542987.1594140	best: 7542987.1594140 (5)	total: 41.7ms	remaining: 27.8ms
6:	learn: 7633499.2037234	test: 7421704.7867090	best: 7421704.7867090 (6)	total: 47.9ms	remaining: 20.5ms
7:	learn: 7516508.7359434	test: 7302410.1690184	best: 7302410.1690184 (7)	total: 54.3ms	remaining: 13.6ms
8:	learn: 7402257.8211472	test: 7185241.8234585	best: 7185241.8234585 (8)	total: 60.7ms	remaining: 6.74ms
9:	learn: 7287622.4179671	test: 7068799.8656834	be

0:	learn: 8381361.9927859	test: 8169021.1839713	best: 8169021.1839713 (0)	total: 5.75ms	remaining: 570ms
1:	learn: 8248366.0438962	test: 8034644.4275664	best: 8034644.4275664 (1)	total: 13.1ms	remaining: 641ms
2:	learn: 8119232.3167760	test: 7905922.0816401	best: 7905922.0816401 (2)	total: 18.8ms	remaining: 609ms
3:	learn: 7993289.4507316	test: 7780828.6612690	best: 7780828.6612690 (3)	total: 23.8ms	remaining: 571ms
4:	learn: 7873026.6075462	test: 7664095.3963142	best: 7664095.3963142 (4)	total: 29.5ms	remaining: 560ms
5:	learn: 7752355.4526254	test: 7542987.1594140	best: 7542987.1594140 (5)	total: 33.4ms	remaining: 523ms
6:	learn: 7633499.2037234	test: 7421704.7867090	best: 7421704.7867090 (6)	total: 37.6ms	remaining: 500ms
7:	learn: 7516508.7359434	test: 7302410.1690184	best: 7302410.1690184 (7)	total: 42ms	remaining: 483ms
8:	learn: 7402257.8211472	test: 7185241.8234585	best: 7185241.8234585 (8)	total: 46.3ms	remaining: 468ms
9:	learn: 7287622.4179671	test: 7068799.8656834	best: 706

81:	learn: 3526825.5551067	test: 3293449.2228695	best: 3293449.2228695 (81)	total: 406ms	remaining: 89.2ms
82:	learn: 3509646.8798901	test: 3277607.5005810	best: 3277607.5005810 (82)	total: 416ms	remaining: 85.1ms
83:	learn: 3491270.3345747	test: 3263866.4699447	best: 3263866.4699447 (83)	total: 433ms	remaining: 82.5ms
84:	learn: 3475677.1810640	test: 3247695.7471976	best: 3247695.7471976 (84)	total: 442ms	remaining: 78ms
85:	learn: 3457862.1901710	test: 3232420.8373748	best: 3232420.8373748 (85)	total: 446ms	remaining: 72.7ms
86:	learn: 3440499.2904398	test: 3218342.3007833	best: 3218342.3007833 (86)	total: 450ms	remaining: 67.3ms
87:	learn: 3425238.0368591	test: 3205175.8393880	best: 3205175.8393880 (87)	total: 455ms	remaining: 62ms
88:	learn: 3410610.9267868	test: 3195145.6514914	best: 3195145.6514914 (88)	total: 460ms	remaining: 56.9ms
89:	learn: 3396177.3444514	test: 3183034.8524018	best: 3183034.8524018 (89)	total: 465ms	remaining: 51.7ms
90:	learn: 3382419.0747296	test: 3169167.

59:	learn: 2939287.5657053	test: 2809812.1856736	best: 2809812.1856736 (59)	total: 299ms	remaining: 200ms
60:	learn: 2930599.2783218	test: 2806460.1503429	best: 2806460.1503429 (60)	total: 305ms	remaining: 195ms
61:	learn: 2922855.5168786	test: 2799463.2147271	best: 2799463.2147271 (61)	total: 310ms	remaining: 190ms
62:	learn: 2914581.4974675	test: 2790826.9639756	best: 2790826.9639756 (62)	total: 315ms	remaining: 185ms
63:	learn: 2906919.7631714	test: 2782512.1306990	best: 2782512.1306990 (63)	total: 322ms	remaining: 181ms
64:	learn: 2900879.0994894	test: 2778423.5704212	best: 2778423.5704212 (64)	total: 327ms	remaining: 176ms
65:	learn: 2892160.6481389	test: 2775250.6026387	best: 2775250.6026387 (65)	total: 331ms	remaining: 170ms
66:	learn: 2883380.5456844	test: 2769035.5570540	best: 2769035.5570540 (66)	total: 337ms	remaining: 166ms
67:	learn: 2876861.7247086	test: 2762044.4271578	best: 2762044.4271578 (67)	total: 342ms	remaining: 161ms
68:	learn: 2872044.0654128	test: 2757395.99812

...

27:	learn: 2681713.6014933	test: 2830921.9995959	best: 2830921.9995959 (27)	total: 695ms	remaining: 4.27s
28:	learn: 2661824.3760670	test: 2821386.6366939	best: 2821386.6366939 (28)	total: 721ms	remaining: 4.25s
29:	learn: 2639116.0303561	test: 2808942.0893170	best: 2808942.0893170 (29)	total: 744ms	remaining: 4.21s
30:	learn: 2616642.5999644	test: 2798459.2535289	best: 2798459.2535289 (30)	total: 767ms	remaining: 4.18s
31:	learn: 2595644.5094858	test: 2785743.2840406	best: 2785743.2840406 (31)	total: 791ms	remaining: 4.15s
32:	learn: 2577166.6913224	test: 2779917.4943335	best: 2779917.4943335 (32)	total: 815ms	remaining: 4.12s
33:	learn: 2559610.1635078	test: 2774472.7512634	best: 2774472.7512634 (33)	total: 838ms	remaining: 4.09s
34:	learn: 2542862.3992704	test: 2768065.0965892	best: 2768065.0965892 (34)	total: 861ms	remaining: 4.06s
35:	learn: 2526234.9040926	test: 2757207.9524798	best: 2757207.9524798 (35)	total: 886ms	remaining: 4.03s
36:	learn: 2511999.4519185	test: 2747493.43832

108:	learn: 2136880.1981258	test: 2622783.9761733	best: 2622783.9761733 (108)	total: 2.67s	remaining: 2.23s
109:	learn: 2130671.1140632	test: 2616726.3165220	best: 2616726.3165220 (109)	total: 2.7s	remaining: 2.21s
110:	learn: 2126048.1375175	test: 2616826.7464405	best: 2616726.3165220 (109)	total: 2.72s	remaining: 2.18s
111:	learn: 2125152.5624112	test: 2616726.4246312	best: 2616726.3165220 (109)	total: 2.74s	remaining: 2.15s
112:	learn: 2120709.0759291	test: 2617105.5415175	best: 2616726.3165220 (109)	total: 2.77s	remaining: 2.13s
113:	learn: 2118034.8054091	test: 2616556.2367031	best: 2616556.2367031 (113)	total: 2.79s	remaining: 2.1s
114:	learn: 2114750.9643001	test: 2614973.7413446	best: 2614973.7413446 (114)	total: 2.81s	remaining: 2.08s
115:	learn: 2113709.7612340	test: 2614777.7066562	best: 2614777.7066562 (115)	total: 2.84s	remaining: 2.05s
116:	learn: 2109365.6545091	test: 2611705.7662000	best: 2611705.7662000 (116)	total: 2.86s	remaining: 2.03s
117:	learn: 2105866.9257881	te

189:	learn: 1911126.8451920	test: 2572294.2473004	best: 2572041.4902466 (187)	total: 4.64s	remaining: 244ms
190:	learn: 1908809.5225247	test: 2572412.5957563	best: 2572041.4902466 (187)	total: 4.67s	remaining: 220ms
191:	learn: 1907010.1797097	test: 2572077.7403018	best: 2572041.4902466 (187)	total: 4.7s	remaining: 196ms
192:	learn: 1902481.4209693	test: 2571002.9720317	best: 2571002.9720317 (192)	total: 4.72s	remaining: 171ms
193:	learn: 1899821.0155644	test: 2572247.5057808	best: 2571002.9720317 (192)	total: 4.74s	remaining: 147ms
194:	learn: 1896821.0446948	test: 2572367.1363884	best: 2571002.9720317 (192)	total: 4.77s	remaining: 122ms
195:	learn: 1896323.6368441	test: 2572399.9045568	best: 2571002.9720317 (192)	total: 4.79s	remaining: 97.7ms
196:	learn: 1894740.7056112	test: 2572452.3803823	best: 2571002.9720317 (192)	total: 4.81s	remaining: 73.3ms
197:	learn: 1892351.5750011	test: 2572560.4070294	best: 2571002.9720317 (192)	total: 4.84s	remaining: 48.9ms
198:	learn: 1886875.724893

74:	learn: 2212542.1247413	test: 2709274.1032920	best: 2709274.1032920 (74)	total: 1.84s	remaining: 3.07s
75:	learn: 2209948.1602633	test: 2707828.1822915	best: 2707828.1822915 (75)	total: 1.87s	remaining: 3.05s
76:	learn: 2205073.2170461	test: 2705238.1890737	best: 2705238.1890737 (76)	total: 1.89s	remaining: 3.02s
77:	learn: 2201561.1913183	test: 2703950.2447672	best: 2703950.2447672 (77)	total: 1.92s	remaining: 3s
78:	learn: 2197680.0096455	test: 2703035.9787562	best: 2703035.9787562 (78)	total: 1.94s	remaining: 2.97s
79:	learn: 2194059.0940407	test: 2702356.5386978	best: 2702356.5386978 (79)	total: 1.96s	remaining: 2.94s
80:	learn: 2191637.1408121	test: 2701469.9346781	best: 2701469.9346781 (80)	total: 1.99s	remaining: 2.92s
81:	learn: 2187974.0274642	test: 2702053.7653377	best: 2701469.9346781 (80)	total: 2.01s	remaining: 2.89s
82:	learn: 2184163.9129181	test: 2700225.2904531	best: 2700225.2904531 (82)	total: 2.03s	remaining: 2.87s
83:	learn: 2183094.1818521	test: 2700632.3997705	

155:	learn: 1974187.1061661	test: 2632479.9285907	best: 2632479.9285907 (155)	total: 3.8s	remaining: 1.07s
156:	learn: 1970642.6882216	test: 2631640.9000027	best: 2631640.9000027 (156)	total: 3.83s	remaining: 1.05s
157:	learn: 1965779.6206134	test: 2628975.1731785	best: 2628975.1731785 (157)	total: 3.85s	remaining: 1.02s
158:	learn: 1962347.7948414	test: 2628123.8811829	best: 2628123.8811829 (158)	total: 3.88s	remaining: 1s
159:	learn: 1959281.9869416	test: 2628084.0210616	best: 2628084.0210616 (159)	total: 3.9s	remaining: 975ms
160:	learn: 1958988.7775844	test: 2627802.4481077	best: 2627802.4481077 (160)	total: 3.92s	remaining: 951ms
161:	learn: 1955789.3597376	test: 2627212.7579550	best: 2627212.7579550 (161)	total: 3.95s	remaining: 926ms
162:	learn: 1950993.9636121	test: 2626262.2863638	best: 2626262.2863638 (162)	total: 3.97s	remaining: 902ms
163:	learn: 1950742.6946895	test: 2626081.7385618	best: 2626081.7385618 (163)	total: 3.99s	remaining: 877ms
164:	learn: 1948304.8022391	test:

In [125]:
result

{'params': {'depth': 7, 'iterations': 200, 'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
    

In [124]:
from catboost import CatBoostRegressor
boost = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7, iterations=200, learning_rate=0.1)
boost.fit(X_train, y_train)
boost_pred = boost.predict(X_test)
print(mean_squared_error(boost_pred, y_test)**0.5)

0:	learn: 4371502.1919520	total: 23.2ms	remaining: 4.62s
1:	learn: 4169155.9023503	total: 45.1ms	remaining: 4.47s
2:	learn: 3986201.7429308	total: 66.7ms	remaining: 4.38s
3:	learn: 3811830.6381156	total: 88.5ms	remaining: 4.34s
4:	learn: 3670974.9242244	total: 111ms	remaining: 4.32s
5:	learn: 3540870.3726754	total: 132ms	remaining: 4.28s
6:	learn: 3433127.3109933	total: 154ms	remaining: 4.25s
7:	learn: 3339180.3637036	total: 176ms	remaining: 4.22s
8:	learn: 3252515.6902292	total: 198ms	remaining: 4.19s
9:	learn: 3176752.6809973	total: 219ms	remaining: 4.16s
10:	learn: 3108249.0347521	total: 252ms	remaining: 4.33s
11:	learn: 3049202.6801832	total: 275ms	remaining: 4.3s
12:	learn: 2989075.2712974	total: 296ms	remaining: 4.26s
13:	learn: 2938044.7662089	total: 318ms	remaining: 4.22s
14:	learn: 2895832.0960200	total: 340ms	remaining: 4.19s
15:	learn: 2859819.4213953	total: 362ms	remaining: 4.16s
16:	learn: 2825929.0989057	total: 384ms	remaining: 4.13s
17:	learn: 2794267.6744933	total: 406m

152:	learn: 2057379.9044620	total: 3.65s	remaining: 1.12s
153:	learn: 2054975.0306432	total: 3.68s	remaining: 1.1s
154:	learn: 2051900.9621735	total: 3.72s	remaining: 1.08s
155:	learn: 2048338.6429742	total: 3.75s	remaining: 1.06s
156:	learn: 2044216.6396477	total: 3.77s	remaining: 1.03s
157:	learn: 2040673.1230289	total: 3.79s	remaining: 1.01s
158:	learn: 2036096.2195738	total: 3.81s	remaining: 983ms
159:	learn: 2033521.0859394	total: 3.83s	remaining: 959ms
160:	learn: 2031508.2665069	total: 3.86s	remaining: 935ms
161:	learn: 2031303.0354401	total: 3.88s	remaining: 911ms
162:	learn: 2031109.1508547	total: 3.9s	remaining: 886ms
163:	learn: 2029470.7759707	total: 3.93s	remaining: 862ms
164:	learn: 2028211.9184511	total: 3.96s	remaining: 839ms
165:	learn: 2026345.0843956	total: 3.99s	remaining: 817ms
166:	learn: 2023983.3461322	total: 4.01s	remaining: 793ms
167:	learn: 2019015.3445762	total: 4.04s	remaining: 769ms
168:	learn: 2017252.1516350	total: 4.06s	remaining: 744ms
169:	learn: 2015

Ситуация аналогична лесу, требуется более скрупулезный перебор, на который нужно время...

In [150]:
from catboost import CatBoostRegressor
boost = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7,
                          iterations=500, learning_rate=0.15, verbose=False)
boost.fit(X_train, y_train)
boost_pred = boost.predict(X_test)
print(mean_squared_error(boost_pred, y_test)**0.5)

2571387.9225949896


Методом научного тыка удалось немного улучшить параметры, результат превзошел модель "из коробки".

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [ ]:
X_train.loc[:, "price"] = y_train
invest = X_train.loc[X_train["product_type_Investment"] == 1, :]
invest_X_train = invest.drop(columns="price")
invest_y_train = invest["price"]
owner = X_train.loc[X_train["product_type_OwnerOccupier"] == 1, :]
owner_X_train = owner.drop(columns="price")
owner_y_train = owner["price"]

In [164]:
boost_invest = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7,
                          iterations=500, learning_rate=0.15, verbose=False)
boost_invest.fit(invest_X_train, invest_y_train)
boost_invest_pred = boost_invest.predict(X_test[X_test["product_type_Investment"] == 1])

boost_owner = CatBoostRegressor(loss_function="RMSE", random_state=42, depth=7,
                          iterations=500, learning_rate=0.15, verbose=False)
boost_owner.fit(owner_X_train, owner_y_train)
boost_owner_pred = boost_owner.predict(X_test[X_test["product_type_OwnerOccupier"] == 1])

In [181]:
boost_invest_pred_indexed = pd.Series(boost_invest_pred, index=X_test[X_test["product_type_Investment"] == 1].index)
boost_owner_pred_indexed = pd.Series(boost_owner_pred, index=X_test[X_test["product_type_OwnerOccupier"] == 1].index)
prediction = pd.concat((boost_invest_pred_indexed, boost_owner_pred_indexed))
prediction = prediction[y_test.index]
print(mean_squared_error(prediction, y_test)**0.5)

2543698.88206796


Как видно, модели обученные на стратифицированных данных (следовательно, на выборкахх с меньшей дисперсией) показали себя лучше, чем одна модель

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [211]:
from sklearn.linear_model import HuberRegressor
huber = HuberRegressor(max_iter=3000).fit(X=invest_X_train, y=invest_y_train)
huber_pred = huber.predict(X_test[X_test["product_type_Investment"] == 1])

In [212]:
rmses = []
for huber_w in np.arange(0, 1.1, 0.1):
    boost_w = 1 - huber_w
    total_pred = huber_pred * huber_w + boost_invest_pred * boost_w
    y_test_invest = y_test[X_test["product_type_Investment"] == 1]
    y_test_invest = y_test_invest[X_test[X_test["product_type_Investment"] == 1].index]
    rmse = mean_squared_error(total_pred, y_test_invest)**0.5
    print(rmse)
    rmses.append(rmse)

3002620.874375416
8355234.900689829
15927970.612124339
23678822.28575164
31476484.576652482
39293098.84970635
47119234.2103081
54950822.80879392
62785824.049745336
70623102.13586074
78461974.79303826


Huber regression - линейная модель с повышенной устойчивочтью к выбросам. Как видно, она катастрофически плохо описывает данные. Вероятнее всего необходим другой подход к подготовке данных, feature engineering.